# Deep Learning

### Tensor

In [8]:
Tensor = list

In [9]:
from typing import List

def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    
    return sizes

In [10]:
print(shape([[23,4,5], [45,55]]))
print(shape([[[12,4,5], [32,5,6], [4,56,24]]]))

[2, 3]
[1, 3, 3]


In [11]:
def is_1d(tensor: Tensor) -> bool:
    '''
    If tensor[0] is a list, its a higher-order tensor.
    Otherwise, tensor is 1-dimensional (that is, a vector).'''
    return not isinstance(tensor[0], list)

In [12]:
print(is_1d([1,3,4]))
print(is_1d([[12,3], [325,5]]))

True
False


In [13]:
def tensor_sum(tensor: Tensor) -> float:
    '''sums up all the values in the tensor'''
    if is_1d(tensor):
        return sum(tensor)
    else:
        return sum(tensor_sum(tensor_i)
                   for tensor_i in tensor)

In [14]:
print(tensor_sum([[1,2,4], [1,3,4], [1,34,5]]))

55


In [15]:
from typing import Callable

In [16]:
def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    '''appliers f elementwise'''
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

In [17]:
print(tensor_apply(lambda x: x+1, [1,2,3]))

print(tensor_apply(lambda x: 2*x, [[1,2], [3, 4]]))

[2, 3, 4]
[[2, 4], [6, 8]]


In [18]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _ : 0.0, tensor)

In [19]:
print(zeros_like([1,2,4]))
print(zeros_like([[12,4,5], [12,4,5], [35,5, 4]]))

[0.0, 0.0, 0.0]
[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


In [20]:
def tensor_combine(f: Callable[[float, float], float],
                   t1: Tensor,
                   t2: Tensor) -> Tensor:
    '''applies f to corresponding elements of t1 and t2'''
    if is_1d(t1):
        return [f(x, y) for x, y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i)
                for t1_i, t2_i in zip(t1, t2)]

In [21]:
import operator

In [22]:
print(tensor_combine(operator.add, [1,2,3], [4,5,6]));print()
print(tensor_combine(operator.mul, [1,2,3], [4,5,6]))

[5, 7, 9]

[4, 10, 18]


## The Layer Abstraction

In [23]:
from typing import Iterable, Tuple

In [24]:
class Layer:
    '''our neural network will be composed of Layers, each of which knows how to do some 
    computation on its inputs in the "forward" direction and propagate gradients in the
    "backward", direction'''
    def forward(self, input):
        '''Note the lack of types. 
        We re not going to be prescriptive about what kinds of inputs layers can take 
        and what kinds of ouputs they can return '''
        
        raise NotImplementedError
    
    def backward(self, gradient):
        """
        Similarly, we're not going to be prescriptive about what the
        gradient looks like. It's up to you the user to make sure
        that you're doing things sensibly.
        """
        raise NotImplementedError

    def params(self) -> Iterable[Tensor]:
        """
        Returns the parameters of this layer. The default implementation
        returns nothing, so that if you have a layer with no parameters
        you don't have to implement this.
        """
        return ()

    def grads(self) -> Iterable[Tensor]:
        """
        Returns the gradients, in the same order as params()
        """
        return ()

In [25]:
from neural_network import sigmoid

In [26]:
class Sigmoid(Layer):
    
    def forward(self, input: Tensor) -> Tensor:
        '''Apply Sigmoid to each element of the input tensor,
        and save the result to use in backpropagration'''
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
    
    def backwark(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad, self.sigmoids, gradient)

# The Linear Layer